<a href="https://colab.research.google.com/github/IlfGauhnith/TM/blob/main/Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download pt_core_news_lg

2023-07-13 04:36:34.233618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 04:36:35.232126: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [27]:
import pandas as pd

df = pd.read_csv("forum.csv")
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Disciplina,Autor,Postagem,Sentimento (0 - negativo; 5 - neutro; 10 - positivo),"Classe (pergunta, resposta, neutro)",Assunto da postagem,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Estatística Descritiva,professor,Mais uma vez sejam bem vindos a disciplina de ...,5,Neutro,NaN,NaN,NaN,NaN
1,Estatística Descritiva,professor,Como estão os estudos? Já começaram a ver o ma...,5,Pergunta,NaN,NaN,NaN,NaN
2,Estatística Descritiva,aluno1,Bom dia. Sim. Pra ser mais preciso comecei a a...,5,Neutro,NaN,NaN,NaN,NaN
3,Estatística Descritiva,aluno2,Iniciei hoje meus estudos nessa matéria !!! A ...,8,resposta,Introdução à Estatística descritiva,NaN,NaN,NaN
4,Estatística Descritiva,professor,Expliquem com suas palavras os conceitos: 1 - ...,5,Pergunta,Introdução à Estatística descritiva,Medidas de tendência central,Medidas de Posição,Medidas de dispersão


# Pré Processamento

In [28]:
import pt_core_news_lg
nlp = pt_core_news_lg.load()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
def preprocess(text):
  text = text.lower()

  res = []
  for token in nlp(text):
    if token.is_stop or token.is_punct:
      continue
    elif token.like_num or token.is_currency:
      continue
    elif len(token.text) < 4:
      continue

    res.append(token.text)

  return ' '.join(res)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
df['Postagem'] = df['Postagem'].apply(preprocess)
df['Postagem']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0      sejam vindos disciplina análise dados vamos co...
1                             estudos começaram material
2                   preciso comecei assistir vídeo aulas
3      iniciei hoje estudos matéria estatística conju...
4      expliquem palavras conceitos distribuição freq...
                             ...                        
128    aluno podemos método amostragem definido taman...
129    método amostragem conveniente permite selecion...
130    população conjunto indivíduos objetos apresent...
131    teorema central limite importante resultado es...
132    pesquisa tendenciosa representaria contraste e...
Name: Postagem, Length: 133, dtype: object

In [31]:
import re

df['Postagem'] = df['Postagem'].apply(lambda text: re.sub(r"aluno\d{1,}", "aluno", text)) # Retirando dígitos após a palavra 'aluno'

df['Postagem'] = df['Postagem'].apply(lambda text: re.sub(r"(?:https?:\/\/)+.+", "", text)) # Retirando links

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Gensim LDA

In [32]:
from gensim.corpora import Dictionary
from gensim.models import Phrases

docs = df['Postagem'].apply(lambda text: text.split(' ')).values

# Add bigrams and trigrams to docs (only ones that appear 12 times or more).
bigram = Phrases(docs, min_count=12)

for idx in range(len(docs)):
  for token in bigram[docs[idx]]:
    if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

# Create a dictionary representation of the documents.
# docs = [' '.join(tokens) for tokens in tokens_nltk]
dictionary = Dictionary(docs)


# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
# Que argumentos são adequados para o caso?
# dictionary.filter_extremes(no_below=20, no_above=0.5)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 4
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
model.show_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.024*"amostra" + 0.017*"população" + 0.013*"valores" + 0.012*"média" + 0.009*"resultados" + 0.008*"hipótese" + 0.008*"amostragem" + 0.008*"aleatória" + 0.008*"será" + 0.007*"estimativa"'),
 (1,
  '0.021*"variável" + 0.019*"regressão" + 0.016*"valores" + 0.015*"parâmetro" + 0.013*"amostra" + 0.013*"estimativa" + 0.012*"variáveis" + 0.012*"populacional" + 0.012*"média" + 0.011*"estatística"'),
 (2,
  '0.023*"dados" + 0.016*"medidas" + 0.015*"população" + 0.014*"amostra" + 0.013*"distribuição" + 0.010*"variáveis" + 0.009*"conjunto" + 0.009*"frequência" + 0.008*"amostragem" + 0.008*"central"'),
 (3,
  '0.017*"amostra" + 0.015*"dados" + 0.015*"população" + 0.015*"distribuição" + 0.013*"medidas" + 0.012*"amostragem" + 0.010*"variáveis" + 0.010*"valores" + 0.009*"central" + 0.009*"linear"')]

In [35]:
import pyLDAvis.gensim_models
import pyLDAvis

pyLDAvis.enable_notebook()

panel = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
panel

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.094139 -0.049369       1        1  35.135035
3     -0.071237  0.006852       2        1  28.501028
1      0.124921 -0.069606       3        1  19.033559
0      0.040454  0.112123       4        1  17.330378, topic_info=             Term       Freq      Total Category  logprob  loglift
449     regressão  27.000000  27.000000  Default  30.0000  30.0000
403      variável  24.000000  24.000000  Default  29.0000  29.0000
710     parâmetro  12.000000  12.000000  Default  28.0000  28.0000
711  populacional  10.000000  10.000000  Default  27.0000  27.0000
688    estimativa  18.000000  18.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
943     contínuas   2.886306   8.121558   Topic4  -5.5588   0.7182
7           dados   3.048584  62.502916   Topic4  -5.5041  -1.2678
70   distribuição   2.889905  44.376200   Topic4  -5.5575  -0.9788
315     elementos   2.889241  18.553336   Topic4  -5.5578  -0.1069
693       pontual   2.889162  11.874671   Topic4  -5.5578   0.3393

[288 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
519       1  0.753517          -preço
335       4  0.719087        acredito
732       3  0.932957       afirmação
822       4  0.719087       aleatoria
1000      2  0.754015  aleatoriamente
...     ...       ...             ...
403       4  0.082302        variável
805       3  0.932957      verdadeiro
564       1  0.945902          visual
718       2  0.871407           ótimo
712       3  0.941909           único

[388 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])